In [2]:
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import requests
import openpyxl
from tqdm import tqdm

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Open the Excel file and select the sheet you want to work with
wb = openpyxl.load_workbook('/Users/rentsher/Desktop/Gumlet/traffic.xlsx')
sheet = wb['Sheet6']

# Get the total number of rows in the sheet
num_rows = sheet.max_row

# Define a function to fetch data for a given batch of URLs
def fetch_data(urls):
    # Initialize a counter variable to keep track of the number of rows processed
    counter = 0

    # Iterate over the URLs in the batch and fetch the data for each URL
    for url in urls:
        # Get the row number for the current URL
        row_number = urls.index(url) + 1

        # Send an HTTP request to the website and parse the HTML content
        logging.debug(f"Fetching data for URL: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the element with the data-testid attribute you are interested in
        element = soup.find(attrs={"data-testid": "siteheader_monthlyvisits"})

        # Extract the data you are interested in from the element
        if element:
            data = element.text
        else:
            data = ""

        # Write the data to the corresponding cell in the Excel sheet
        sheet.cell(row=row_number, column=2).value = data

        # Increment the counter variable
        counter += 1

    # Return the number of rows processed

# Define the batch size
batch_size = 500

# Initialize a list to hold the URLs
urls = []

# Iterate over the rows in the sheet and fetch the URLs
for row in sheet.iter_rows(min_row=2, max_col=1):
    # Get the URL from cell A
    url = row[0].value

    # Add the URL to the list
    urls.append(url)

# Split the list of URLs into batches
url_batches = [urls[i:i+batch_size] for i in range(0, len(urls), batch_size)]

# Initialize a counter variable to keep track of the number of rows processed
counter = 0

# Use ThreadPoolExecutor to fetch data for each batch of URLs
with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit the tasks to the executor and track their progress with tqdm
    futures = [executor.submit(fetch_data, batch) for batch in url_batches]
    for future in tqdm(as_completed(futures), total=len(url_batches), desc="Fetching data", unit="batch"):
        # Get the result of the task and increment the counter variable
        result = future.result()
        counter += result

# Save the changes to the Excel file
wb.save('/Users/rentsher/Desktop/Gumlet/traffic.xlsx')

# Print the percentage of data fetched
print(f"{counter / num_rows * 100:.2f}% of data fetched")


2023-04-11 17:30:44,414 - DEBUG - Fetching data for URL: https://www.similarsites.com/site/glu.nl
Fetching data:   0%|                                   | 0/1 [00:00<?, ?batch/s]2023-04-11 17:30:44,651 - DEBUG - Starting new HTTPS connection (1): www.similarsites.com:443
2023-04-11 17:30:45,850 - DEBUG - https://www.similarsites.com:443 "GET /site/glu.nl HTTP/1.1" 200 None
2023-04-11 17:30:45,889 - DEBUG - Fetching data for URL: https://www.similarsites.com/site/yc.edu
2023-04-11 17:30:45,893 - DEBUG - Starting new HTTPS connection (1): www.similarsites.com:443
2023-04-11 17:30:46,990 - DEBUG - https://www.similarsites.com:443 "GET /site/yc.edu HTTP/1.1" 200 None
2023-04-11 17:30:47,028 - DEBUG - Fetching data for URL: https://www.similarsites.com/site/tower-london.com
2023-04-11 17:30:47,032 - DEBUG - Starting new HTTPS connection (1): www.similarsites.com:443
2023-04-11 17:30:48,108 - DEBUG - https://www.similarsites.com:443 "GET /site/tower-london.com HTTP/1.1" 200 None
2023-04-11 1

TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'